# Flexible Coordinator Workflow

This notebook demonstrates a flexible text-to-SQL workflow where the coordinator makes intelligent decisions about which agent to run based on the current state of the query tree.

## Key Features

1. **Non-linear Execution**: Coordinator decides which agent to call based on node state
2. **Automatic Node Management**: Current node is tracked in memory
3. **State-based Decisions**: Coordinator examines what each node needs
4. **Error Recovery**: Can retry specific steps without restarting
5. **Complex Query Support**: Handles multi-node query trees intelligently

## Workflow Logic

The coordinator examines the current node and decides:
- No intent? → Run query_analyzer
- No mapping? → Run schema_linker
- No SQL? → Run sql_generator
- No execution/evaluation? → Run sql_evaluator
- Poor quality? → Retry the appropriate step
- All good? → Check for workflow completion

In [1]:
import os
import sys
import asyncio
import logging
from pathlib import Path
from typing import Dict, Any, List, Optional
from dotenv import load_dotenv

sys.path.append('../src')
load_dotenv()

# Check for API key
if not os.getenv("OPENAI_API_KEY"):
    print("WARNING: OPENAI_API_KEY not found in environment")
else:
    print("✓ OPENAI_API_KEY found")

# Set up logging
logging.basicConfig(level=logging.INFO, 
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Reduce noise
logging.getLogger('autogen_core').setLevel(logging.WARNING)
logging.getLogger('httpx').setLevel(logging.WARNING)

✓ OPENAI_API_KEY found


## 1. Import All Components

In [2]:
# Memory and managers
from keyvalue_memory import KeyValueMemory
from task_context_manager import TaskContextManager
from query_tree_manager import QueryTreeManager
from database_schema_manager import DatabaseSchemaManager
from node_history_manager import NodeHistoryManager

# Schema reader
from schema_reader import SchemaReader

# All 4 agents
from query_analyzer_agent import QueryAnalyzerAgent
from schema_linker_agent import SchemaLinkerAgent
from sql_generator_agent import SQLGeneratorAgent
from sql_evaluator_agent import SQLEvaluatorAgent

# Memory types
from memory_content_types import (
    TaskContext, QueryNode, NodeStatus, TaskStatus,
    QueryMapping, TableMapping, ColumnMapping, JoinMapping,
    TableSchema, ColumnInfo, ExecutionResult
)

# AutoGen components
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [ ]:
# Initialize shared memory
memory = KeyValueMemory()

# Initialize managers
task_manager = TaskContextManager(memory)
tree_manager = QueryTreeManager(memory)
schema_manager = DatabaseSchemaManager(memory)
history_manager = NodeHistoryManager(memory)

print("✓ Initialized memory and managers")

In [ ]:
# Initialize OpenAI client for coordinator
coordinator_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    temperature=0.1,
    timeout=120,
    api_key=os.getenv("OPENAI_API_KEY")
)

# Create flexible coordinator
coordinator = AssistantAgent(
    name="coordinator",
    system_message="""You are a flexible coordinator for a text-to-SQL workflow.

Your agents:
- query_analyzer: Analyzes queries and creates query trees
- schema_linker: Links queries to database schema
- sql_generator: Generates SQL from linked schema
- sql_evaluator: Executes and evaluates SQL results

HOW TO CALL AGENTS:
- query_analyzer: Call with the user's query directly
- Other agents: Just call with a simple task description like:
  - "Link query to database schema"
  - "Generate SQL query"
  - "Analyze SQL execution results"
  
The agents automatically work on the current node stored in memory.

DECISION PROCESS:

1. First, always check the logs to understand the current state:
   - Look for "current node" mentions
   - Check what components the node already has
   - Check the quality of any evaluations

2. Based on the current node's state, decide what to do:
   - If no query tree exists → call query_analyzer with the user's query
   - If node lacks mapping → call schema_linker with "Link query to database schema"  
   - If node lacks SQL → call sql_generator with "Generate SQL query"
   - If node lacks evaluation → call sql_evaluator with "Analyze SQL execution results"
   - If evaluation quality is poor → analyze the issue and retry appropriate step

3. After sql_evaluator runs:
   - Check if it says "workflow complete" in the logs
   - If workflow is complete → summarize the final answer and say "TERMINATE"
   - If not complete → continue with the next node

4. TERMINATION RULES:
   - You MUST say "TERMINATE" at the end of your message when workflow is complete
   - Look for "workflow complete" in the sql_evaluator logs
   - When all nodes have good quality results, summarize and say "TERMINATE"
   - Example: "The highest rate is 75.0%. TERMINATE"

5. For poor quality results:
   - Wrong tables? → retry schema_linker with guidance
   - Bad SQL? → retry sql_generator with error details
   - Missing data? → check if schema linking was complete

CRITICAL: You must end your final answer with the word "TERMINATE" to end the conversation.""",
    model_client=coordinator_client,
    tools=[query_analyzer.get_tool(), schema_linker.get_tool(), 
           sql_generator.get_tool(), sql_evaluator.get_tool()]
)

print("✓ Created flexible coordinator with termination rules")

async def display_current_state():
    """Display the current workflow state"""
    print("\n" + "="*60)
    print("CURRENT WORKFLOW STATE")
    print("="*60)
    
    # Current node
    current_node_id = await memory.get("current_node_id")
    print(f"\nCurrent Node: {current_node_id or 'None'}")
    
    # Workflow status
    is_complete = await memory.get("workflow_complete")
    print(f"Workflow Complete: {'Yes' if is_complete else 'No'}")
    
    # Tree overview
    tree = await tree_manager.get_tree()
    if tree and "nodes" in tree:
        print(f"\nQuery Tree:")
        print(f"  Total nodes: {len(tree['nodes'])}")
        
        # Count by status
        status_counts = {}
        for node_id, node_data in tree["nodes"].items():
            status = node_data.get("status", "unknown")
            status_counts[status] = status_counts.get(status, 0) + 1
        
        for status, count in status_counts.items():
            print(f"  {status}: {count}")

async def check_workflow_completion():
    """Check if workflow is truly complete and provide summary"""
    is_complete = await memory.get("workflow_complete")
    tree = await tree_manager.get_tree()
    
    if is_complete:
        print("\n✅ WORKFLOW MARKED AS COMPLETE")
    
    if tree and "nodes" in tree:
        all_good = True
        results_summary = []
        
        for node_id, node_data in tree["nodes"].items():
            if node_data.get("sql") and node_data.get("executionResult"):
                analysis = await memory.get(f"node_{node_id}_analysis")
                if analysis:
                    quality = analysis.get("result_quality", "unknown")
                    if quality not in ["excellent", "good"]:
                        all_good = False
                    
                    # Collect results
                    exec_result = node_data["executionResult"]
                    if exec_result.get("data") and len(exec_result["data"]) > 0:
                        results_summary.append({
                            "intent": node_data.get("intent", ""),
                            "result": exec_result["data"][0] if exec_result["data"] else None,
                            "quality": quality
                        })
        
        if all_good:
            print("✅ All nodes have good quality results")
        else:
            print("⚠️  Some nodes still need improvement")
        
        if results_summary:
            print("\n📊 Results Summary:")
            for item in results_summary:
                print(f"  • {item['intent'][:50]}...")
                print(f"    Result: {item['result']}")
                print(f"    Quality: {item['quality']}")
    
    return is_complete

async def display_node_details(node_id: str):
    """Display detailed information about a specific node"""
    node = await tree_manager.get_node(node_id)
    if not node:
        print(f"Node {node_id} not found")
        return
    
    print(f"\nNode: {node_id}")
    print(f"  Status: {node.status.value if node.status else 'None'}")
    print(f"  Intent: {node.intent[:50]}..." if node.intent else "  Intent: None")
    print(f"  Has mapping: {'Yes' if node.mapping else 'No'}")
    print(f"  Has SQL: {'Yes' if node.sql else 'No'}")
    print(f"  Has execution: {'Yes' if node.executionResult else 'No'}")
    
    # Check evaluation
    analysis = await memory.get(f"node_{node_id}_analysis")
    if analysis:
        print(f"  Evaluation:")
        print(f"    Answers intent: {analysis.get('answers_intent')}")
        print(f"    Quality: {analysis.get('result_quality')}")

async def display_progress():
    """Display workflow progress"""
    tree = await tree_manager.get_tree()
    if not tree or "nodes" not in tree:
        print("No query tree found")
        return
    
    print("\n" + "="*60)
    print("WORKFLOW PROGRESS")
    print("="*60)
    
    current_node_id = await memory.get("current_node_id")
    
    for node_id, node_data in tree["nodes"].items():
        is_current = "→" if node_id == current_node_id else " "
        
        # Build status indicators
        indicators = []
        if node_data.get("intent"):
            indicators.append("I")
        if node_data.get("mapping"):
            indicators.append("M")
        if node_data.get("sql"):
            indicators.append("S")
        if node_data.get("executionResult"):
            indicators.append("E")
        
        # Check evaluation
        analysis = await memory.get(f"node_{node_id}_analysis")
        if analysis:
            quality = analysis.get("result_quality", "?")[0].upper()
            indicators.append(f"Q:{quality}")
        
        status_str = "["+",".join(indicators)+"]" if indicators else "[empty]"        
        intent_preview = node_data.get("intent", "No intent")[:40] + "..."
        
        print(f"{is_current} {node_id[-8:]} {status_str} {intent_preview}")
    
    print("\nLegend: I=Intent, M=Mapping, S=SQL, E=Executed, Q=Quality")

In [4]:
# Database configuration
data_path = "/home/norman/work/text-to-sql/MAC-SQL/data/bird"
tables_json_path = Path(data_path) / "dev_tables.json"
db_name = "california_schools"

# Test queries
test_queries = [
    "What is the highest eligible free rate for K-12 students in schools located in Alameda County?",
    "Show me schools with SAT scores above 1400 and their free lunch eligibility rates",
    "Find the top 5 counties by average SAT scores, including the number of schools and average free lunch rate"
]

# Pick a query (try different ones!)
test_query = test_queries[0]
print(f"Query: {test_query}")
print("-" * 80)

# Initialize task
task_id = "flexible_demo_001"
await task_manager.initialize(task_id, test_query, db_name)

# Load schema
schema_reader = SchemaReader(
    data_path=data_path,
    tables_json_path=str(tables_json_path),
    dataset_name="bird",
    lazy=False
)

await schema_manager.load_from_schema_reader(schema_reader, db_name)

# Get schema summary
summary = await schema_manager.get_schema_summary()
print(f"\nLoaded '{db_name}' database:")
print(f"  Tables: {summary['table_count']}")
print(f"  Columns: {summary['total_columns']}")
print(f"  Foreign keys: {summary['total_foreign_keys']}")

2025-05-25 12:27:08,441 - TaskContextManager - INFO - Initialized task context for task flexible_demo_001


Query: What is the highest eligible free rate for K-12 students in schools located in Alameda County?
--------------------------------------------------------------------------------
load json file from /home/norman/work/text-to-sql/MAC-SQL/data/bird/dev_tables.json

Loading all database info...
Found 11 databases in bird dataset


2025-05-25 12:27:20,951 - DatabaseSchemaManager - INFO - Initialized empty database schema
2025-05-25 12:27:20,952 - DatabaseSchemaManager - INFO - Added table 'frpm' to schema
2025-05-25 12:27:20,952 - DatabaseSchemaManager - INFO - Added table 'satscores' to schema
2025-05-25 12:27:20,953 - DatabaseSchemaManager - INFO - Added table 'schools' to schema
2025-05-25 12:27:20,953 - DatabaseSchemaManager - INFO - Loaded schema for database 'california_schools' with 3 tables



Loaded 'california_schools' database:
  Tables: 3
  Columns: 89
  Foreign keys: 2


# Process messages and show coordinator decisions
step_count = 0
max_steps = 50  # Safety limit to prevent infinite loops

async for message in stream:
    if hasattr(message, 'source') and message.source == 'coordinator':
        step_count += 1
        print(f"\n[Step {step_count}] Coordinator:")
        
        if hasattr(message, 'content'):
            if isinstance(message.content, list) and len(message.content) > 0:
                # Tool calls
                for tool_call in message.content:
                    if hasattr(tool_call, 'name'):
                        print(f"  → Calling {tool_call.name}")
            elif isinstance(message.content, str):
                # Show decision reasoning
                preview = message.content[:200] + "..." if len(message.content) > 200 else message.content
                print(f"  Decision: {preview}")
        
        # Safety check for max steps
        if step_count >= max_steps:
            print(f"\n⚠️  Reached maximum steps ({max_steps}). Stopping to prevent infinite loop.")
            print("The workflow may not have completed properly.")
            break

print("\n" + "="*80)
print("WORKFLOW COMPLETE")
print("="*80)

In [5]:
# LLM configuration
llm_config = {
    "model_name": "gpt-4o",
    "temperature": 0.1,
    "timeout": 60
}

# Initialize all agents
query_analyzer = QueryAnalyzerAgent(memory, llm_config)
schema_linker = SchemaLinkerAgent(memory, llm_config)
sql_generator = SQLGeneratorAgent(memory, llm_config)
sql_evaluator = SQLEvaluatorAgent(memory, llm_config)

print("✓ Initialized all agents")

2025-05-25 12:27:20,975 - QueryAnalyzerAgent - INFO - Initialized query_analyzer with model gpt-4o
2025-05-25 12:27:20,986 - SchemaLinkerAgent - INFO - Initialized schema_linker with model gpt-4o
2025-05-25 12:27:20,997 - SQLGeneratorAgent - INFO - Initialized sql_generator with model gpt-4o
2025-05-25 12:27:21,008 - SQLEvaluatorAgent - INFO - Initialized sql_evaluator with model gpt-4o


✓ Initialized all agents


In [ ]:
# Check if workflow actually completed
is_complete = await check_workflow_completion()

if not is_complete:
    print("\n⚠️  Workflow did not complete properly!")
    print("The coordinator may have failed to detect completion.")
    print("Check the logs above for 'workflow complete' messages.")

## 5. Create Flexible Coordinator

This coordinator examines the current state and makes intelligent decisions about what to do next.

In [6]:
# Initialize OpenAI client for coordinator
coordinator_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    temperature=0.1,
    timeout=120,
    api_key=os.getenv("OPENAI_API_KEY")
)

# Create flexible coordinator
coordinator = AssistantAgent(
    name="coordinator",
    system_message="""You are a flexible coordinator for a text-to-SQL workflow.

Your agents:
- query_analyzer: Analyzes queries and creates query trees
- schema_linker: Links queries to database schema
- sql_generator: Generates SQL from linked schema
- sql_evaluator: Executes and evaluates SQL results

DECISION PROCESS:

1. First, always check the logs to understand the current state:
   - Look for "current node" mentions
   - Check what components the node already has
   - Check the quality of any evaluations

2. Based on the current node's state, decide what to do:
   - If no query tree exists → call query_analyzer
   - If node lacks mapping → call schema_linker
   - If node lacks SQL → call sql_generator
   - If node lacks evaluation → call sql_evaluator
   - If evaluation quality is poor → analyze the issue and retry appropriate step

3. After sql_evaluator runs:
   - Check if it says "workflow complete" → summarize and TERMINATE
   - Check if it moved to a new node → examine that node's needs
   - Check if quality was poor → decide how to fix it

4. For poor quality results, consider:
   - Wrong tables? → retry schema_linker with guidance
   - Bad SQL? → retry sql_generator with error details
   - Missing data? → check if schema linking was complete

IMPORTANT:
- Agents automatically work on the current node in memory
- sql_evaluator manages node progression automatically
- Only TERMINATE when you see "workflow complete"
- Be smart about retries - understand what went wrong first""",
    model_client=coordinator_client,
    tools=[query_analyzer.get_tool(), schema_linker.get_tool(), 
           sql_generator.get_tool(), sql_evaluator.get_tool()]
)

print("✓ Created flexible coordinator")

✓ Created flexible coordinator


## 6. Helper Functions

In [7]:
async def display_current_state():
    """Display the current workflow state"""
    print("\n" + "="*60)
    print("CURRENT WORKFLOW STATE")
    print("="*60)
    
    # Current node
    current_node_id = await memory.get("current_node_id")
    print(f"\nCurrent Node: {current_node_id or 'None'}")
    
    # Workflow status
    is_complete = await memory.get("workflow_complete")
    print(f"Workflow Complete: {'Yes' if is_complete else 'No'}")
    
    # Tree overview
    tree = await tree_manager.get_tree()
    if tree and "nodes" in tree:
        print(f"\nQuery Tree:")
        print(f"  Total nodes: {len(tree['nodes'])}")
        
        # Count by status
        status_counts = {}
        for node_id, node_data in tree["nodes"].items():
            status = node_data.get("status", "unknown")
            status_counts[status] = status_counts.get(status, 0) + 1
        
        for status, count in status_counts.items():
            print(f"  {status}: {count}")

async def display_node_details(node_id: str):
    """Display detailed information about a specific node"""
    node = await tree_manager.get_node(node_id)
    if not node:
        print(f"Node {node_id} not found")
        return
    
    print(f"\nNode: {node_id}")
    print(f"  Status: {node.status.value if node.status else 'None'}")
    print(f"  Intent: {node.intent[:50]}..." if node.intent else "  Intent: None")
    print(f"  Has mapping: {'Yes' if node.mapping else 'No'}")
    print(f"  Has SQL: {'Yes' if node.sql else 'No'}")
    print(f"  Has execution: {'Yes' if node.executionResult else 'No'}")
    
    # Check evaluation
    analysis = await memory.get(f"node_{node_id}_analysis")
    if analysis:
        print(f"  Evaluation:")
        print(f"    Answers intent: {analysis.get('answers_intent')}")
        print(f"    Quality: {analysis.get('result_quality')}")

async def display_progress():
    """Display workflow progress"""
    tree = await tree_manager.get_tree()
    if not tree or "nodes" not in tree:
        print("No query tree found")
        return
    
    print("\n" + "="*60)
    print("WORKFLOW PROGRESS")
    print("="*60)
    
    current_node_id = await memory.get("current_node_id")
    
    for node_id, node_data in tree["nodes"].items():
        is_current = "→" if node_id == current_node_id else " "
        
        # Build status indicators
        indicators = []
        if node_data.get("intent"):
            indicators.append("I")
        if node_data.get("mapping"):
            indicators.append("M")
        if node_data.get("sql"):
            indicators.append("S")
        if node_data.get("executionResult"):
            indicators.append("E")
        
        # Check evaluation
        analysis = await memory.get(f"node_{node_id}_analysis")
        if analysis:
            quality = analysis.get("result_quality", "?")[0].upper()
            indicators.append(f"Q:{quality}")
        
        status_str = "["+"".join(indicators)+"]" if indicators else "[empty]"        
        intent_preview = node_data.get("intent", "No intent")[:40] + "..."
        
        print(f"{is_current} {node_id[-8:]} {status_str} {intent_preview}")
    
    print("\nLegend: I=Intent, M=Mapping, S=SQL, E=Executed, Q=Quality")

## 7. Run the Flexible Workflow

In [8]:
# Create team with termination condition
termination_condition = TextMentionTermination("TERMINATE")
team = RoundRobinGroupChat(
    participants=[coordinator],
    termination_condition=termination_condition
)

print("Starting flexible workflow...\n")
stream = team.run_stream(task=test_query)

Starting flexible workflow...



In [ ]:
# Process messages and show coordinator decisions
step_count = 0
async for message in stream:
    if hasattr(message, 'source') and message.source == 'coordinator':
        step_count += 1
        print(f"\n[Step {step_count}] Coordinator:")
        
        if hasattr(message, 'content'):
            if isinstance(message.content, list) and len(message.content) > 0:
                # Tool calls
                for tool_call in message.content:
                    if hasattr(tool_call, 'name'):
                        print(f"  → Calling {tool_call.name}")
            elif isinstance(message.content, str):
                # Show decision reasoning
                preview = message.content[:200] + "..." if len(message.content) > 200 else message.content
                print(f"  Decision: {preview}")

print("\n" + "="*80)
print("WORKFLOW COMPLETE")
print("="*80)


[Step 1] Coordinator:
  → Calling query_analyzer


2025-05-25 12:27:27,098 - QueryTreeManager - INFO - Initialized query tree with root node node_1748190447.098254_root
2025-05-25 12:27:27,098 - NodeHistoryManager - INFO - Added create operation for node node_1748190447.098254_root
2025-05-25 12:27:27,098 - QueryTreeManager - INFO - Added node node_1748190447.098875_1 to tree
2025-05-25 12:27:27,099 - NodeHistoryManager - INFO - Added create operation for node node_1748190447.098875_1
2025-05-25 12:27:27,099 - QueryTreeManager - INFO - Added node node_1748190447.099315_2 to tree
2025-05-25 12:27:27,099 - NodeHistoryManager - INFO - Added create operation for node node_1748190447.099315_2
2025-05-25 12:27:27,099 - QueryTreeManager - INFO - Updated node node_1748190447.098254_root
2025-05-25 12:27:27,099 - QueryAnalyzerAgent - WARNING - Complex query but no children found - set root as current
2025-05-25 12:27:27,099 - QueryAnalyzerAgent - INFO - Query analysis completed. Complexity: complex. Root node: node_1748190447.098254_root



[Step 2] Coordinator:
  → Calling query_analyzer

[Step 3] Coordinator:
  Decision: {"messages": [{"source": "user", "models_usage": null, "metadata": {}}, {"source": "query_analyzer", "models_usage": {"prompt_tokens": 3165, "completion_tokens": 316}, "metadata": {}}], "stop_reason":...

[Step 4] Coordinator:
  → Calling schema_linker


2025-05-25 12:27:30,170 - QueryTreeManager - INFO - Updated node node_1748190447.098254_root
2025-05-25 12:27:30,170 - NodeHistoryManager - INFO - Added revise operation for node node_1748190447.098254_root
2025-05-25 12:27:30,171 - SchemaLinkerAgent - INFO - Updated node node_1748190447.098254_root with schema mapping



[Step 5] Coordinator:
  → Calling schema_linker

[Step 6] Coordinator:
  Decision: {"messages": [{"source": "user", "models_usage": null, "metadata": {}}, {"source": "schema_linker", "models_usage": {"prompt_tokens": 5032, "completion_tokens": 217}, "metadata": {}}], "stop_reason": ...

[Step 7] Coordinator:
  → Calling sql_generator


...


[Step 8] Coordinator:
  → Calling sql_generator

[Step 9] Coordinator:
  Decision: {"messages": [{"source": "user", "models_usage": null, "metadata": {}}, {"source": "sql_generator", "models_usage": {"prompt_tokens": 421, "completion_tokens": 170}, "metadata": {}}], "stop_reason": n...


2025-05-25 12:27:36,696 - SQLEvaluatorAgent - ERROR - No node_id found in task. Task must be in format 'node:{node_id} - ...'



[Step 10] Coordinator:
  → Calling sql_evaluator


2025-05-25 12:27:38,056 - SQLEvaluatorAgent - ERROR - No evaluation XML found in output



[Step 11] Coordinator:
  → Calling sql_evaluator

[Step 12] Coordinator:
  Decision: {"messages": [{"source": "user", "models_usage": null, "metadata": {}}, {"source": "sql_evaluator", "models_usage": {"prompt_tokens": 302, "completion_tokens": 51}, "metadata": {}}], "stop_reason": nu...

[Step 13] Coordinator:
  Decision: The workflow is complete. The highest eligible free rate for K-12 students in schools located in Alameda County has been successfully determined.

[Step 14] Coordinator:
  Decision: The highest eligible free rate for K-12 students in schools located in Alameda County has been successfully determined. If you need further details or have another query, feel free to ask!

[Step 15] Coordinator:
  Decision: The highest eligible free rate for K-12 students in schools located in Alameda County has been successfully determined. If you need further details or have another query, feel free to ask!

[Step 16] Coordinator:
  Decision: The highest eligible free rate for K-12 st